3. Directory Read Use Cases
Read all tower logs using: Path glob filter (example: *.csv) Multiple paths input Recursive lookup

Demonstrate these 3 reads separately: Using pathGlobFilter Using list of paths in spark.read.csv([path1, path2]) Using .option("recursiveFileLookup","true")

Compare the outputs and understand when each should be used.

In [0]:
Tower_log_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower", header=True, inferSchema=True, pathGlobFilter="*.csv", recursiveFileLookup=True, sep="|")
display(Tower_log_df)

# spark.read.format(csv).option("header", "true").option("inferSchema", "true").option("recursiveFileLookup","true").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower").display()



In [0]:
tower_log_df1 = spark.read.csv(["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/", "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower_all_data/"], header=True, inferSchema=True, recursiveFileLookup=True, pathGlobFilter="*.csv", sep="|")
display(tower_log_df1)

##4. Schema Inference, Header, and Separator
1. Try the Customer, Usage files with the option and options using read.csv and format function:<br>
header=false, inferSchema=false<br>
or<br>
header=true, inferSchema=true<br>
2. Write a note on What changed when we use header or inferSchema  with true/false?<br>
3. How schema inference handled “abc” in age?<br>

In [0]:
customer_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer", header=True, inferSchema=True).printSchema()
display(customer_df)

#When Header is false and inferschema is false data format is taken as string, 
#When Header is true and inferschema is true is data format is taken as data format given in data file
#when either header or inferschema is false, data format is taken as string

# 3. How schema inference handled “abc” in age?
# Age contains both integer and string value so it is considerign the age column as string[string + integer = string]


##5. Column Renaming Usecases
1. Apply column names using string using toDF function for customer data
2. Apply column names and datatype using the schema function for usage data
3. Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data 

In [0]:
dbutils.fs.rm("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer1/", True)
#custid,name,age,city,plan
customer1_data = '''
1,"divya",20,"bangalore",postpaid
2,"anu",24,"chennai",postpaid
3,"sita",25,"hyderabad",postpaid
'''
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer1/")
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer1/customer1.csv", customer1_data, True)

customer_df1 = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer1/customer1.csv",header=False, inferSchema=True).toDF("customer_id","user_name","age","city","plan")
#if we user todf() function no need to use of header = true
# display(customer_df1)

customer_df1.write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer1/customer_csv_folder")

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, DoubleType, TimestampType
usage_data_schema = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("voice_mins", IntegerType(), False),
    StructField("data_mb", IntegerType(), False),
    StructField("sms_count", IntegerType(), False),
])

usage_df = spark.read.schema(usage_data_schema).options(header="True", sep="\t").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv")
#while using schema we need to use options() function

display(usage_df)

In [0]:
usage_data_schema = StructType([
    StructField("event_id", IntegerType(), True),
    StructField("customer_id", LongType(), False),
    StructField("tower_id", StringType(), False),
    StructField("signal_strength", LongType(), False),
    StructField("timestamp", TimestampType(), False)
])

tower_df1 = spark.read.schema(usage_data_schema).options(header="True", sep="|").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/tower_logs_region1.csv")
display(tower_df1)